In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import ecephys_analyses as ea
import seaborn as sns

In [3]:
def normalize(df):
    df = df.copy()
    levels = df.index.names
    df = df / df.loc['baseline-light-period', 'NREM'] * 100
    df = df.reorder_levels(levels).sort_index()
    return df

In [4]:
keep_subjects = ["Segundo", "Valentino", "Doppio", "Alessandro", "Allan"]

# SWA

In [5]:
pwr = pd.read_csv(ea.package_datapath('mean-bandpower.csv'))
pwr = pwr.set_index(['condition', 'state', 'region', 'band', 'subject']).sort_index()
pwr

power
condition             state          region      band  subject                  
baseline-light-period NREM           mpta_wm_ref delta Alessandro  208874.597587
                                                       Allan       328467.363411
                                                       Doppio      225727.755399
                                                       Eugene      257810.423571
                                                       Segundo     248743.670119
...                                                                          ...
recovery              all-SPW-states sr_wm_ref   theta Allan       164334.485991
                                                       Doppio      210346.050331
                                                       Eugene      257641.290143
                                                       Segundo     324421.450582
                                                       Valentino   251716.216176

[864 rows x 1 columns]

In [6]:
pwr = normalize(pwr)
pwr

power
condition             state          region      band  subject               
baseline-light-period NREM           mpta_wm_ref delta Alessandro  100.000000
                                                       Allan       100.000000
                                                       Doppio      100.000000
                                                       Eugene      100.000000
                                                       Segundo     100.000000
...                                                                       ...
recovery              all-SPW-states sr_wm_ref   theta Allan        97.323856
                                                       Doppio      101.892368
                                                       Eugene      109.602396
                                                       Segundo     103.784464
                                                       Valentino   103.946736

[864 rows x 1 columns]

In [7]:
def get_swa_rebound_and_decline(pwr, state):
    rebounds = pwr.loc['first2h-recovery'] - pwr.loc['first2h-recovery-match']
    declines = pwr.loc['early-recovery'] - pwr.loc['late-recovery']

    df = pd.concat(
        [
            rebounds.loc[state, "mpta_wm_ref", "delta"],
            declines.loc[state, "mpta_wm_ref", "delta"],
            rebounds.loc[state, "sr_wm_ref", "delta"],
            declines.loc[state, "sr_wm_ref", "delta"],
        ],
        keys=["cx_rebound", "cx_decline", "hc_rebound", "hc_decline"],
        axis=1,
    ).droplevel(1, axis=1)
    return df

swa_nrem = get_swa_rebound_and_decline(pwr, 'NREM').loc[keep_subjects]

In [8]:
swa_nrem

,cx_rebound,cx_decline,hc_rebound,hc_decline
subject,,,,
Segundo,174.665864,126.744205,42.015040,36.053249
Valentino,103.253351,77.368286,35.941385,25.159800
Doppio,157.877901,99.269911,43.654388,39.583519
Alessandro,108.873443,66.366003,40.002798,49.034242
Allan,189.005556,166.388974,33.788480,35.245754
